In [1]:
import math
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import numpy.ma as ma
import pandas as pd
import datetime as dt
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors 
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import cartopy.crs as ccrs
import cartopy.crs as crs
import cartopy.feature as cfeature
import h5py  
from astropy.time import Time
import os
import glob
from osgeo import gdal
import scipy.ndimage
import rasterio as rio
import re
import geopandas as gpd
import earthpy as et
import earthpy.plot as ep
import earthpy.spatial as es
import earthpy.mask as em
from pyhdf.SD import SD, SDC
import pprint
import csv

In [2]:
# Set input directory, and change working directory - plug in D:
inDir = "D:\\masters_data\\ATL08"   # I should change this so I can work from github?
os.chdir(inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(inDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [3]:
# constants for conversion lat lon -> tile H,V and r,c
R = 6371007.181	    # the radius of the idealizes sphere representing the Earth
T = 1111950	        # the height and width of each MODIS tile in the projection plane
Xmin = -20015109    # the western limit of the projection plane
Ymax = 10007555	    # the northern limit of the projection plane
w = T/2400.0	    # the actual size of a '500m' MODIS sinusoidal grid cell. (would be /1200 for a 1km grid)

In [4]:
all_points = pd.read_csv("D:\\masters_data\\output\\csv\\all_points.csv", header=None)
all_points.columns= ["latitude", "longitude", "mean_canopy", "max_canopy", "min_canopy", "median_canopy", "canopy_openness" ]

In [5]:
latitude = all_points['latitude'][:]
longitude = all_points['longitude'][:]
mean_height = all_points["mean_canopy"][:]
max_height = all_points["max_canopy"][:]
min_height = all_points["min_canopy"][:]
median_canopy = all_points["median_canopy"][:]
canopy_openness = all_points["canopy_openness"][:]

In [6]:
string_latitude = latitude.to_string(index = False)
string_latitude = latitude.replace("\n", ",")

string_longitude = longitude.to_string(index = False)
string_longitude = longitude.replace("\n", ",")

string_mean_height = mean_height.to_string(index = False)
string_mean_height = mean_height.replace("\n", ",")

string_max_height = max_height.to_string(index = False)
string_max_height = max_height.replace("\n", ",")

string_min_height = min_height.to_string(index = False)
string_min_height = min_height.replace("\n", ",")

string_median_canopy = median_canopy.to_string(index = False)
string_median_canopy = median_canopy.replace("\n", ",")

string_canopy_openness = canopy_openness.to_string(index = False)
string_canopy_openness = canopy_openness.replace("\n", ",")

In [7]:
float_latitude = [float(i) for i in string_latitude]
float_longitude = [float(i) for i in string_longitude]
float_mean_height = [float(i) for i in string_mean_height]
float_max_height = [float(i) for i in string_max_height]
float_min_height = [float(i) for i in string_min_height]
float_median_canopy = [float(i) for i in string_median_canopy]
float_canopy_openness = [float(i) for i in string_canopy_openness]

In [8]:
canopy_height_array = np.zeros((2400,2400))
canopy_max_height_array = np.zeros((2400,2400))
canopy_min_height_array = np.zeros((2400,2400))
canopy_median_height_array = np.zeros((2400,2400))
canopy_openness_array = np.zeros((2400,2400))

In [9]:
number = 0
r_list = []
c_list = []
    
for i in latitude:
    lat = float_latitude[number]
    lon = float_longitude[number]
    phi = lat/(180/math.pi) # convert to radians
    lamb = lon/(180/math.pi)
    x = R*lamb*math.cos(phi)
    y = R*phi
    r = int(math.floor(((Ymax-y)%T)/w-0.5))
    c = int(math.floor(((x-Xmin)%T)/w-0.5))
    
    r_list.append(r)
    c_list.append(c)
    number += 1

In [10]:
for ii in range(len(r_list)):
    row = r_list[ii]
    col = c_list[ii]
    canopy_height_array[row, col] = float_mean_height[ii]
    canopy_max_height_array[row, col] = float_max_height[ii]
    canopy_min_height_array[row, col] = float_min_height[ii]
    canopy_median_height_array[row, col] = float_median_canopy[ii]
    canopy_openness_array[row, col] = float_canopy_openness[ii]

In [11]:
First_BA = pd.read_csv("D:\\masters_data\\output\\first_BA.csv", header=None)
Second_BA = pd.read_csv("D:\\masters_data\\output\\second_BA.csv", header=None)

In [12]:
Mask = np.zeros((2400, 2400))
# low_low / low_med / low_high

Mask[((First_BA<1) * (Second_BA<1))==1] = 1
Mask[((First_BA<1) * (Second_BA>=1) * (Second_BA<7))==1] = 2
Mask[((First_BA<1) * (Second_BA>=7))==1] = 3

# med_low / med_med / med_high
Mask[((First_BA>=1) * (First_BA<7) * (Second_BA<1))==1] = 4
Mask[((First_BA>=1) * (First_BA<7) * (Second_BA>=1) * (Second_BA<7))==1] = 5
Mask[((First_BA>=1) * (First_BA<7) * (Second_BA>=7))==1] = 6

#high_low / high_med / high_high
Mask[((Second_BA<1) * (First_BA>=7))==1] = 7
Mask[((First_BA>=7) * (Second_BA>=1) * (Second_BA<7))==1] = 8
Mask[((First_BA>=7) * (Second_BA>=7))==1] = 9

# this takes out where canopy height == 0, but means that the colormaps are hardly visible
# Mask[(canopy_height_array==0)==1]=0

In [13]:
First_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\first_BA_500-1500_H20V08.csv", header=None)
Second_BA_500_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\second_BA_500-1500_H20V08.csv", header=None)
First_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\first_BA_sub500_H20V08.csv", header=None)
Second_BA_sub500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\second_BA_sub500_H20V08.csv", header=None)
First_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\first_BA_above1500_H20V08.csv", header=None)
Second_BA_above_1500 = pd.read_csv("D:\\masters_data\\rainfall\\Rainfall_bands\\H20_V08\\second_BA_above1500_H20V08.csv", header=None)

In [14]:
rain_band_mask = np.zeros((2400, 2400))

rain_band_mask[(First_BA_sub500) != 0] = 1
rain_band_mask[(Second_BA_sub500) != 0] = 1

rain_band_mask[(First_BA_500_1500) != 0] = 2
rain_band_mask[(Second_BA_500_1500) != 0] = 2

rain_band_mask[(First_BA_above_1500) != 0] = 3
rain_band_mask[(Second_BA_above_1500) != 0] = 3

In [15]:
canopy_height_list = canopy_height_array.flatten().tolist()
canopy_max_height_list = canopy_max_height_array.flatten().tolist()
canopy_min_height_list = canopy_min_height_array.flatten().tolist()
canopy_median_height_list = canopy_median_height_array.flatten().tolist()
canopy_openness_list = canopy_openness_array.flatten().tolist()
BA_list = Mask.flatten().tolist()
rain_band_list = rain_band_mask.flatten().tolist() 

In [44]:
all_data = pd.DataFrame({'mean_canopy': canopy_height_list})

In [17]:
rain_band_list = map(str, rain_band_list)
rain_band_list = [i.replace('1.0', 'sub500') for i in rain_band_list]
rain_band_list = [i.replace('2.0', '500_1500') for i in rain_band_list]
rain_band_list = [i.replace('3.0', 'above1500') for i in rain_band_list]

In [18]:
BA_list = map(str, BA_list)
BA_list = [i.replace('1.0', 'low_low') for i in BA_list]
BA_list = [i.replace('2.0', 'low_med') for i in BA_list]
BA_list = [i.replace('3.0', 'low_high') for i in BA_list]
BA_list = [i.replace('4.0', 'med_low') for i in BA_list]
BA_list = [i.replace('5.0', 'med_med') for i in BA_list]
BA_list = [i.replace('6.0', 'med_high') for i in BA_list]
BA_list = [i.replace('7.0', 'high_low') for i in BA_list]
BA_list = [i.replace('8.0', 'high_med') for i in BA_list]
BA_list = [i.replace('9.0', 'high_high') for i in BA_list]

In [45]:
all_data['canopy_max_height']= canopy_max_height_list
all_data['canopy_min_height']= canopy_min_height_list
all_data['canopy_median_height']= canopy_median_height_list
all_data['canopy_openness']= canopy_openness_list
all_data['rainfall_band']= rain_band_list
all_data['BA_category']= BA_list

In [46]:
all_data

,mean_canopy,canopy_max_height,canopy_min_height,canopy_median_height,canopy_openness,rainfall_band,BA_category
0,0.0,0.0,0.0,0.0,0.0,sub500,high_med
1,0.0,0.0,0.0,0.0,0.0,sub500,high_high
2,0.0,0.0,0.0,0.0,0.0,sub500,high_high
3,0.0,0.0,0.0,0.0,0.0,sub500,high_high
4,0.0,0.0,0.0,0.0,0.0,sub500,high_high
...,...,...,...,...,...,...,...
5759995,0.0,0.0,0.0,0.0,0.0,500_1500,med_med
5759996,0.0,0.0,0.0,0.0,0.0,sub500,med_low
5759997,0.0,0.0,0.0,0.0,0.0,500_1500,med_low
5759998,0.0,0.0,0.0,0.0,0.0,500_1500,med_low


In [50]:
all_data =all_data[all_data['mean_canopy'] !=0]

In [52]:
all_data['Tile']= "H20V08"

In [53]:
all_data

,mean_canopy,canopy_max_height,canopy_min_height,canopy_median_height,canopy_openness,rainfall_band,BA_category,Tile
23,3.704632,9.813477,0.548615,2.532440,3.019267,sub500,high_high,H20V08
49,2.339680,7.279022,0.530609,1.502518,1.758361,sub500,med_med,H20V08
77,1.919061,9.207825,0.500610,0.895737,2.327772,sub500,med_med,H20V08
82,4.491499,8.381744,0.597687,4.342331,2.661070,sub500,high_med,H20V08
84,2.862178,6.075073,0.514832,2.707748,1.703719,sub500,med_med,H20V08
...,...,...,...,...,...,...,...,...
5759898,3.097404,8.218842,0.611847,1.470428,2.535568,above1500,med_med,H20V08
5759927,1.739094,5.968048,0.520660,1.278107,1.291870,500_1500,med_med,H20V08
5759956,6.402081,12.361481,0.538544,6.703522,3.121775,0.0,low_low,H20V08
5759957,6.361335,11.280823,0.524414,6.636810,2.745829,0.0,low_low,H20V08


In [ ]:
np.savetxt("D:\\masters_data\\output\\csv\\complete_H20_V08_data.csv", complete_data, delimiter=",")